In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import wandb
from pytorch_lightning.loggers import WandbLogger
import torch
from Models.EGformer import EGformer
import torch.nn as nn
import torch.optim as optim
from ocpmodels.datasets import LmdbDataset
from torch.utils.data import random_split
import torch_geometric.loader as geom_loader
import torch_geometric.data as data
from typing import Any
import yaml
import torch.nn.init as init
import torch.nn as nn

c:\Users\lhuang37\.conda\envs\ocp-models\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:Invalid setup for SCN. Either the e3nn library or Jd.pt is missing.


In [2]:
# Save the model hyperparameters to a YAML file
model_hparams ={"num_atoms":0,
                "bond_feat_dim":0,
                "num_targets":0,
                "num_heads":4,
                }

with open('params/model_hparams.yml', 'w') as file:
    yaml.dump(model_hparams, file)

In [64]:
class Egformer(nn.Module):

    def __init__(self,**loaded_model_hparams):
        super(Egformer,self).__init__()
        self.arc=EGformer(**loaded_model_hparams)        
        self.mask_loss = nn.MSELoss()

    def forward(self,data):
        pred_output = self.arc(data)
        masks=data.data.y/data.natoms
        masks=norm_label(masks)
        loss = self.mask_loss(pred_output.view(-1, 1), masks.view(-1, 1))
        acc=abs(pred_output.view(-1, 1)-masks.view(-1, 1))

        if masks !=None:            
            return loss,acc

        return loss,acc
    def norm_label(data,y_mean=-7,y_std=6):

        label=data.y/data.natoms
        label=(label-y_mean)/y_std
        return label

In [65]:
with open('params/model_hparams.yml', 'r') as file:
    loaded_model_hparams = yaml.load(file, Loader=yaml.FullLoader)

# Create the model using the loaded hyperparameters
model = Egformer(**loaded_model_hparams)
checkpoint_path='params/gemnet_oc_base_oc20_oc22.pt'
pretrained_state_dict = torch.load(checkpoint_path)['state_dict']
new_model_state_dict = model.arc.state_dict()
filtered_pretrained_state_dict = {k.strip('module.module.'): v for k, v in pretrained_state_dict.items() if k.strip('module.module.') in new_model_state_dict}
new_model_state_dict.update(filtered_pretrained_state_dict)
model.arc.load_state_dict(new_model_state_dict)
for param_name, param in model.named_parameters():

    if param_name.replace('arc.','') in filtered_pretrained_state_dict.keys():
        
        param.requires_grad = False

c:\Users\lhuang37\.conda\envs\ocp-models\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [66]:
y_mean=-7
y_std=6
num_epochs=30
batch_size = 2
learning_rate=0.01
CHECKPOINT_PATH="./checkpoints"
DEVICE='cuda'
dataset=LmdbDataset({"src":"Data/bulks.lmdb"})
criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [67]:
f_paras,t_paras=0,0
for param_name,param in model.named_parameters():
    if param.requires_grad is False:
        f_paras+=1
    else:
        t_paras+=1
print('Freeze params is',f_paras)
print('Need optimiz params is',t_paras)

Freeze params is 238
Need optimiz params is 131


In [68]:
train_length = int(0.8 * len(dataset))
val_length = len(dataset) - train_length

# Split the dataset into train and validation
train_dataset, val_dataset =random_split(dataset, [train_length, val_length])
train_loader = geom_loader.DataLoader(train_dataset, batch_size=1)
val_loader = geom_loader.DataLoader(val_dataset, batch_size=1)

In [69]:
data = next(iter(train_loader))
batched_data = data.batch
print(batched_data.shape)

torch.Size([56])


In [70]:
from tqdm import tqdm

In [71]:
def train_fn(data_loader,model,optimizer):
    model.train()
    total_loss=0.0
    
    for images in tqdm(data_loader):
        images=images.to(DEVICE)        
        optimizer.zero_grad()
        loss,acc=model(images)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        
    return total_loss/len(data_loader)

In [72]:
def eval_fn(data_loader,model):
    model.eval()
    total_loss=0.0
    total_acc=0    
    with torch.no_grad():
        for images in tqdm(data_loader):
            
            images=images.to(DEVICE)                      
            loss,acc=model(images)
            total_loss+=loss.item()
            total_acc+=acc.item()

    return total_loss/len(data_loader),total_acc/len(data_loader)

In [73]:
best_valid_loss=np.Inf
for i in range(num_epochs):
    train_loss=train_fn(train_loader,model,optimizer)
    valid_loss,acc=eval_fn(valid_loader,model)
    print(acc)
    
    if valid_loss< best_valid_loss:
        torch.save(model.state_dict(),'best_model_n.pt')
        print('saved-model')
        best_valid_loss=valid_loss

    print(f'epoch:{i+1} Train_loss:{train_loss} Valid_loss:{valid_loss} Valid acc:{acc}')

  0%|          | 0/3785 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!